In [ ]:
import clean
import pandas as pd
from tqdm import tqdm
import ollama

初始化数据和ollama客户端

In [ ]:
original_data = clean.data()
cleaned_data = original_data.create_save()
client = ollama.Client()

 设置批处理大小和实验参数

In [ ]:
batch_size = 100
limited_data = cleaned_data.head(100)  # 使用.head(100)限制数据量; 如果要处理所有数据，直接使用cleaned_data
model_list = ['gemma:2b','llama2',  'llama_tem_0.1', 'llama_tem_0.4', 'llama_tem_0.7']  # 定义模型列表
treatment_prompts = [
    "Question:What patient treatment plan described in this text?",
    "Question:Identify the treatment strategy for the patient's diagnosis."
]
age_prompts = [
    "Question:What is the patient's age based on this text?",
    "Question:What is the patient's age based on this text? Tips: If there is a specific age mentioned in the text, just state 'The patient is X years old.' If the age is not explicit but can be inferred, state the possible age range and provide a brief reasoning. If it's not possible to determine, state 'Cannot determine the patient's age.'",
    "Question:What is the patient's age based on this text? tip:Please answer very concisely.example:the patient is 23 years old."
]


进行消融实验

In [ ]:
for model_name in model_list:
    for t_index, t_prompt in enumerate(treatment_prompts, start=1):
        for a_index, a_prompt in enumerate(age_prompts, start=1):
            treatment_summaries = []
            patient_ages = []
            for index, row in tqdm(limited_data.iterrows(), total=limited_data.shape[0]):
                text = row['description'] + " " + row['transcription']

                # 生成治疗计划的总结
                treatment_response = client.generate(model='gemma:2b', prompt=f"{t_prompt}\n{text}")
                treatment_summaries.append(treatment_response['response'])

                # 生成患者年龄的问题
                age_response = client.generate(model='gemma:2b', prompt=f"{a_prompt}\n{text}")
                patient_ages.append(age_response['response'])
            if model_name == 'gemma:2b':
                model_name = 'gemma'
            # 构建文件名以反映模型名称和提示词的使用，并指定保存目录
            filename = f"{save_directory}\\{model_name}_t{t_index}_a{a_index}.csv"
            results_df = pd.DataFrame({
                'Treatment Plan': treatment_summaries,
                'Patient Age': patient_ages
            })
            results_df.to_csv(filename, index=False)